# Mavlink Camera
> Mavlink Camera Component for sending commands to a camera on a companion computer or GCS
> The server is on the companion computer and the client is on the ground station PC.

In [ ]:
#| default_exp mavlink.test_camera

[autoreload of UAV.mavlink.camera failed: Traceback (most recent call last):
  File "/home/jn/PycharmProjects/UAV/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/jn/PycharmProjects/UAV/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/home/jn/PycharmProjects/UAV/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/jn/PycharmProjects/UAV/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 349, in update_class
    if update_generic(old_obj, new_obj):
  File "/home/jn/PycharmProjects/UAV/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/home/jn/PycharmProjects/UAV/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export
import time

from UAV.mavlink.mavcom import MAVCom, time_since_boot_ms, time_UTC_usec, date_time_str
from UAV.mavlink.component import Component, mavutil, mavlink, MAVLink

from UAV.mavlink.camera import *
from fastcore.test import *

In [ ]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *


### Implementation of these commands:
>
> [MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS = 527](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS)
[MAV_CMD_REQUEST_CAMERA_INFORMATION = 523](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION)
[MAV_CMD_REQUEST_CAMERA_SETTINGS = 524](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS)
[MAV_CMD_REQUEST_STORAGE_INFORMATION = 525](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION)
[MAV_CMD_STORAGE_FORMAT = 526](https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT)
[MAV_CMD_SET_CAMERA_ZOOM = 531](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM)
[MAV_CMD_SET_CAMERA_FOCUS = 532](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_FOCUS)
[MAV_CMD_IMAGE_START_CAPTURE = 2000](https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE)
[MAV_CMD_IMAGE_STOP_CAPTURE = 2001](https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE)
> 
> [MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION = 2504](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION)
[MAV_CMD_REQUEST_VIDEO_STREAM_STATUS = 2505](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_STATUS)
[MAV_CMD_VIDEO_START_CAPTURE = 2500](https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_CAPTURE)
[MAV_CMD_VIDEO_STOP_CAPTURE = 2501](https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_CAPTURE)
[MAV_CMD_SET_CAMERA_MODE = 530](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_MODE)
> 
**Note**
The simulated camera is implemented in PX4 [gazebo_camera_manager_plugin.cpp](https://github.com/PX4/PX4-SITL_gazebo-classic/blob/main/src/gazebo_camera_manager_plugin.cpp).


In [ ]:
#| exports
# from pymavlink.dialects.v20 import ardupilotmega as mav
# from pymavlink.dialects.v20.ardupilotmega import MAVLink


NAN = float("nan")

"""
MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS = 527 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS
MAV_CMD_REQUEST_CAMERA_INFORMATION = 521 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION
MAV_CMD_REQUEST_CAMERA_SETTINGS = 522 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS
MAV_CMD_REQUEST_STORAGE_INFORMATION = 525 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION
MAV_CMD_STORAGE_FORMAT = 526 # https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT
MAV_CMD_SET_CAMERA_ZOOM = 531 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM
MAV_CMD_SET_CAMERA_FOCUS = 532 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_FOCUS
MAV_CMD_IMAGE_START_CAPTURE = 2000  # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE
MAV_CMD_IMAGE_STOP_CAPTURE = 2001  # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE
MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION = 2504 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION
MAV_CMD_REQUEST_VIDEO_STREAM_STATUS = 2505 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_STATUS
MAV_CMD_VIDEO_START_CAPTURE = 2500 # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_CAPTURE
MAV_CMD_VIDEO_STOP_CAPTURE = 2501 # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_CAPTURE
MAV_CMD_SET_CAMERA_MODE = 530 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_MODE

"""
CAMERA_INFORMATION = mavlink.MAVLINK_MSG_ID_CAMERA_INFORMATION # https://mavlink.io/en/messages/common.html#CAMERA_INFORMATION
CAMERA_SETTINGS = mavlink.MAVLINK_MSG_ID_CAMERA_SETTINGS # https://mavlink.io/en/messages/common.html#CAMERA_SETTINGS
STORAGE_INFORMATION = mavlink.MAVLINK_MSG_ID_STORAGE_INFORMATION # https://mavlink.io/en/messages/common.html#STORAGE_INFORMATION
CAMERA_CAPTURE_STATUS = mavlink.MAVLINK_MSG_ID_CAMERA_CAPTURE_STATUS # https://mavlink.io/en/messages/common.html#CAMERA_CAPTURE_STATUS
CAMERA_IMAGE_CAPTURED = mavlink.MAVLINK_MSG_ID_CAMERA_IMAGE_CAPTURED # https://mavlink.io/en/messages/common.html#CAMERA_IMAGE_CAPTURED


In [ ]:
show_doc(read_camera_info_from_toml)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### read_camera_info_from_toml

>      read_camera_info_from_toml (toml_file_path)

Read MAVLink camera info from a TOML file.

In [ ]:
show_doc(WaitMessage)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### WaitMessage

>      WaitMessage (target_system, target_component)

Wait for a specific message from the server

In [ ]:
show_doc(WaitMessage.set_condition)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### WaitMessage.set_condition

>      WaitMessage.set_condition (msg_id, target_system, target_component)

Set the condition function to generate event for a specific message received from the server

In [ ]:
show_doc(WaitMessage.get)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### WaitMessage.get

>      WaitMessage.get (timeout=1)

Get the object if the event is set or wait until it's set with an optional timeout.

Returns:
    The object if the event is set, or None if it times out or the event isn't set.

In [ ]:
show_doc(CameraClient)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### CameraClient

>      CameraClient (source_component, mav_type, debug)

Create a Viewsheen mavlink gimbal client component for send commands to a gimbal on a companion computer or GCS

|    | **Details** |
| -- | ----------- |
| source_component | used for component indication |
| mav_type | used for heartbeat MAV_TYPE indication |
| debug | logging level |

In [ ]:
show_doc(CameraServer)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### CameraServer

>      CameraServer (source_component, mav_type, camera, debug)

Create a mavlink Camera server Component using a test GSTREAMER pipeline

|    | **Details** |
| -- | ----------- |
| source_component | used for component indication |
| mav_type | used for heartbeat MAV_TYPE indication |
| camera | camera  (or FakeCamera for testing) |
| debug | logging level |

In [ ]:
MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA

con1, con2 = "udpin:localhost:14445", "udpout:localhost:14445"
# con1, con2 = "/dev/ttyACM0", "/dev/ttyUSB0"
with MAVCom(con1, source_system=111, debug=False) as client:
    with MAVCom(con2, source_system=222, debug=False) as server:
        cam:CameraClient = client.add_component(
            CameraClient(mav_type=MAV_TYPE_GCS, source_component=11, debug=True))
        # server.add_component(CameraServer(mav_type=MAV_TYPE_CAMERA, source_component=22, camera=cam_fake1, debug=False))
        server.add_component(CameraServer(mav_type=MAV_TYPE_CAMERA, source_component=22, camera=None, debug=True))

        cam.wait_heartbeat(target_system=222, target_component=22, timeout=1)
        time.sleep(0.1)
        cam.set_target(222, 22)

   
        msg = cam.request_storage_information()
        print (msg)
        
        time.sleep(1)
        

INFO   | uav.MAVCom      | 52.304 |  mavcom.py:396 | Thread-19 (listen) | MAVLink Mav2: True, source_system: 111
INFO   | uav.MAVCom      | 52.405 |  mavcom.py:396 | Thread-20 (listen) | MAVLink Mav2: True, source_system: 222
DEBUG  | uav.CameraClien | 52.406 | component.py:131 | MainThread         | Called from Component.start_mav_connection(), override to add startup behaviour
INFO   | uav.CameraClien | 52.407 | component.py:127 | MainThread         | Component Started self.source_component = 11, self.mav_type = 6, self.source_system = 111
DEBUG  | uav.CameraServe | 52.407 | component.py:131 | MainThread         | Called from Component.start_mav_connection(), override to add startup behaviour
WARNIN | uav.CameraServe | 52.408 |  camera.py:362 | MainThread         | Component has no camera object
INFO   | uav.CameraServe | 52.408 | component.py:127 | MainThread         | Component Started self.source_component = 22, self.mav_type = 30, self.source_system = 222
DEBUG  | uav.CameraClien

set_mav_connection CameraClient component.py:123 self.mav_connection = <MAVCom>
set_mav_connection CameraServer component.py:123 self.mav_connection = <MAVCom>
None


INFO   | uav.CameraServe | 55.410 | component.py:382 | MainThread         | CameraServer closed
DEBUG  | uav.CameraServe | 55.411 |  camera.py:601 | MainThread         | Closed connection to camera
INFO   | uav.MAVCom      | 55.411 |  mavcom.py:447 | MainThread         | MAVCom  closed
INFO   | uav.CameraClien | 57.412 | component.py:382 | MainThread         | CameraClient closed
INFO   | uav.MAVCom      | 57.413 |  mavcom.py:447 | MainThread         | MAVCom  closed


In [ ]:
# | hide
# assert False, "Stop here"

#### Test locally using UDP ports

#### Starting a client and server
 > on the same machine using UDP ports `14445`  with `server_system_ID=111, client_system_ID=222`

In [ ]:
#| exports
from UAV.mavlink.mavcom import MAVCom
from UAV.mavlink.component import Component, mavutil
import time

MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA

class Cam1(Component):
    def __init__(self, source_component, mav_type, debug=False):
        super().__init__(source_component=source_component, mav_type=mav_type,
                         debug=debug)

class Cam2(Component):
    def __init__(self, source_component, mav_type, debug=False):
        super().__init__(source_component=source_component, mav_type=mav_type,
                         debug=debug)
class Cli(Component):
    def __init__(self, source_component, mav_type, debug=False):
        super().__init__( source_component=source_component, mav_type=mav_type,
                         debug=debug)

In [ ]:
#| exports
def test_ack():
    # Test sending a command and receiving an ack from client to server
    with MAVCom("udpin:localhost:14445", source_system=111, debug=False) as client:
        with MAVCom("udpout:localhost:14445", source_system=222, debug=False) as server:
            client.add_component(Cli( mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
            server.add_component(Cam1( mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
            server.add_component(Cam1( mav_type=MAV_TYPE_CAMERA, source_component = 23, debug=False))
            
            for key, comp in client.component.items():
                if comp.wait_heartbeat(target_system=222, target_component=22, timeout=0.1):
                    print ("*** Received heartbeat **** " )
            NUM_TO_SEND = 2
            for i in range(NUM_TO_SEND):
                client.component[11]._test_command(222, 22, 1)
                client.component[11]._test_command(222, 23, 1)
                
            client.component[11]._test_command(222, 24, 1)
    
        print(f"{server.source_system = };  {server.message_cnts = }")
        print(f"{client.source_system = };  {client.message_cnts = }")
        print()
        print(f"{client.source_system = } \n{client.summary()} \n")
        print(f"{server.source_system = } \n{server.summary()} \n")
    
        assert client.component[11].num_cmds_sent == NUM_TO_SEND * 2 + 1
        assert client.component[11].num_acks_rcvd == NUM_TO_SEND * 2
        assert client.component[11].num_acks_drop == 1
        assert server.component[22].num_cmds_rcvd == NUM_TO_SEND
        assert server.component[23].num_cmds_rcvd == NUM_TO_SEND
test_ack()

INFO   | uav.MAVCom      | 57.560 |  mavcom.py:396 | Thread-25 (listen) | MAVLink Mav2: True, source_system: 111
INFO   | uav.MAVCom      | 57.662 |  mavcom.py:396 | Thread-26 (listen) | MAVLink Mav2: True, source_system: 222
INFO   | uav.Cli         | 57.664 | component.py:127 | MainThread         | Component Started self.source_component = 11, self.mav_type = 6, self.source_system = 111
INFO   | uav.Cam1        | 57.664 | component.py:127 | MainThread         | Component Started self.source_component = 22, self.mav_type = 30, self.source_system = 222
INFO   | uav.Cam1        | 57.666 | component.py:127 | MainThread         | Component Started self.source_component = 23, self.mav_type = 30, self.source_system = 222


set_mav_connection Cli component.py:123 self.mav_connection = <MAVCom>
set_mav_connection Cam1 component.py:123 self.mav_connection = <MAVCom>
set_mav_connection Cam1 component.py:123 self.mav_connection = <MAVCom>
*** Received heartbeat **** 


WARNIN | uav.Cli         | 58.167 | component.py:352 | MainThread         | **No ACK: 222/22 MAV_CMD_DO_DIGICAM_CONTROL:203
WARNIN | uav.Cli         | 58.669 | component.py:352 | MainThread         | **No ACK: 222/23 MAV_CMD_DO_DIGICAM_CONTROL:203
WARNIN | uav.Cli         | 59.172 | component.py:352 | MainThread         | **No ACK: 222/22 MAV_CMD_DO_DIGICAM_CONTROL:203
WARNIN | uav.Cli         | 59.675 | component.py:352 | MainThread         | **No ACK: 222/23 MAV_CMD_DO_DIGICAM_CONTROL:203
ERROR  | uav.MAVCom      | 59.677 |  mavcom.py:419 | Thread-26 (listen) |  Component 24 does not exist? ; Exception: 24
WARNIN | uav.Cli         | 00.177 | component.py:352 | MainThread         | **No ACK: 222/24 MAV_CMD_DO_DIGICAM_CONTROL:203
INFO   | uav.Cam1        | 00.668 | component.py:382 | MainThread         | Cam1 closed
INFO   | uav.Cam1        | 01.669 | component.py:382 | MainThread         | Cam1 closed
INFO   | uav.MAVCom      | 01.670 |  mavcom.py:447 | MainThread         | MAVCom  cl

server.source_system = 222;  server.message_cnts = {111: {'COMMAND_LONG': 5, 'HEARTBEAT': 3}}
client.source_system = 111;  client.message_cnts = {222: {'HEARTBEAT': 7}}

client.source_system = 111 
 - comp.source_component = 11
 - comp.num_msgs_rcvd = 7
 - comp.num_cmds_sent = 5
 - comp.num_cmds_rcvd = 0
 - comp.num_acks_rcvd = 0
 - comp.num_acks_sent = 0
 - comp.num_acks_drop = 5
 - comp.message_cnts = {222: {'HEARTBEAT': 7}} 

server.source_system = 222 
 - comp.source_component = 22
 - comp.num_msgs_rcvd = 5
 - comp.num_cmds_sent = 0
 - comp.num_cmds_rcvd = 2
 - comp.num_acks_rcvd = 0
 - comp.num_acks_sent = 0
 - comp.num_acks_drop = 0
 - comp.message_cnts = {111: {'COMMAND_LONG': 2, 'HEARTBEAT': 3}}
 - comp.source_component = 23
 - comp.num_msgs_rcvd = 5
 - comp.num_cmds_sent = 0
 - comp.num_cmds_rcvd = 2
 - comp.num_acks_rcvd = 0
 - comp.num_acks_sent = 0
 - comp.num_acks_drop = 0
 - comp.message_cnts = {111: {'COMMAND_LONG': 2, 'HEARTBEAT': 3}} 


INFO   | uav.Cli         | 03.669 | component.py:382 | MainThread         | Cli closed
INFO   | uav.MAVCom      | 03.671 |  mavcom.py:447 | MainThread         | MAVCom  closed


AssertionError: 

In [ ]:
#| Hide
# assert False, "Stop here"

In [ ]:
show_doc(Component.set_source_compenent)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/component.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Component.set_source_compenent

>      Component.set_source_compenent ()

Set the source component for the master.mav

In [ ]:
show_doc(Component.send_heartbeat)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/component.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Component.send_heartbeat

>      Component.send_heartbeat ()

Send a heartbeat message to indicate the server is alive.

In [ ]:
show_doc(Component.send_command)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/component.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Component.send_command

>      Component.send_command (target_system:int, target_component:int,
>                              command_id:int, params:list, timeout=0.5)

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| target_system | int |  | target system |
| target_component | int |  | target component |
| command_id | int |  | mavutil.mavlink.MAV_CMD.... |
| params | list |  | list of parameters |
| timeout | float | 0.5 | seconds |

In [ ]:
show_doc(Component.wait_heartbeat)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/component.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Component.wait_heartbeat

>      Component.wait_heartbeat (remote_mav_type=None, target_system=None,
>                                target_component=None, timeout:int=1,
>                                tries:int=3)

Wait for a heartbeat from target_system and target_component.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| remote_mav_type | NoneType | None | type of remote system |
| target_system | NoneType | None | target system |
| target_component | NoneType | None | target component |
| timeout | int | 1 | seconds |
| tries | int | 3 |  |
| **Returns** | **bool** |  | **number of tries** |

In [ ]:
show_doc(Component.wait_ack)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/component.py#LNone){target="_blank" style="float:right; font-size:smaller"}

### Component.wait_ack

>      Component.wait_ack (target_system, target_component, command_id=None,
>                          timeout=0.1)

Wait for an ack from target_system and target_component.

In [ ]:
show_doc(Component.send_ping)

#### Test with Serial ports
Test using a Pixhawk connected via telemetry 2 and USB serial ports.
CamClient is set to udpin:localhost:14445 and CamServer is set to udpout:localhost:14435 udpin is so that the client can receive UDP from the mavproxy server at localhost:14445
mavproxy.py --master=/dev/ttyACM1 --baudrate 57600 --out udpout:localhost:14445 mavproxy.py --master=/dev/ttyACM3 --baudrate 57600 --out udpout:localhost:14435

In [ ]:
# Test sending a command and receiving an ack from client to server
with MAVCom("/dev/ttyACM0", source_system=111, debug=False) as client:
    with MAVCom("/dev/ttyUSB0", source_system=222, debug=False) as server:
        client.add_component(Cli(client, mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 23, debug=False))
        
        for key, comp in client.component.items():
            if comp.wait_heartbeat(target_system=222, target_component=22, timeout=0.1):
                print ("*** Received heartbeat **** " )
        NUM_TO_SEND = 2
        for i in range(NUM_TO_SEND):
            client.component[11]._test_command(222, 22, 1)
            client.component[11]._test_command(222, 23, 1)
            
        client.component[11]._test_command(222, 24, 1)

    print(f"{server.source_system = };  {server.message_cnts = }")
    print(f"{client.source_system = };  {client.message_cnts = }")
    print()
    print(f"{client.source_system = } \n{client.summary()} \n")
    print(f"{server.source_system = } \n{server.summary()} \n")

    assert client.component[11].num_cmds_sent == NUM_TO_SEND * 2 + 1
    assert client.component[11].num_acks_rcvd == NUM_TO_SEND * 2
    assert client.component[11].num_acks_drop == 1
    assert server.component[22].num_cmds_rcvd == NUM_TO_SEND
    assert server.component[23].num_cmds_rcvd == NUM_TO_SEND

> For debugging help see http://localhost:3000/tutorials/mavlink_doc&debug.html and http://localhost:3000/tutorials/mavlink_doc&debug.html#debugging

In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev.nbdev_export()